In [33]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
import os
os.getcwd()

'/home/weihangzhang/AC209---final-project/src'

In [4]:
data = h5py.File("./data_sample.hdf5", "r")
print("data keys: "+str(list(data.keys())))

data keys: ['images', 'labels', 'observation_days']


In [5]:
images = data["images"][:]
print("image shape: (sample, x_size, y_size, epoch) = "+str(images.shape))
labels = data["labels"][:]

image shape: (sample, x_size, y_size, epoch) = (72000, 21, 21, 48)


In [16]:
dic = {'Asteroids':0, 'Constant':1, 'EmptyLigh':2, 'M33Cephei':3, 'RRLyrae':4, 'Supernova':5}
labels_digit = [dic[i] for i in labels]
# print(labels_digit)

# labels_flatten = np.repeat(labels_digit, 48)
enc = OneHotEncoder()
labels_onehot = enc.fit_transform(np.array(labels_digit).reshape(-1, 1))

In [17]:
labels_onehot.shape

(72000, 6)

In [18]:
# flatten_data = images.transpose(0, 3, 1, 2).reshape((72000*48, 21, 21, 1))
images.shape

(72000, 21, 21, 48)

In [21]:
# sample_idx = np.random.choice(range(len(flatten_data)), 100000)
# sample_data = flatten_data[sample_idx]
# sample_labels = labels_flatten_onehot[sample_idx]
# sample_data.shape

In [24]:
# x_train, x_val, y_train, y_val = train_test_split(sample_data, sample_labels)

In [25]:
# x_train.shape, y_train.shape

((75000, 21, 21, 1), (75000, 6))

In [99]:
def init_model():
    model = Sequential()
    
    model.add(Conv2D(48, (4,4), input_shape=(21,21,48), padding='same', activation='relu', kernel_initializer='uniform'))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.5))
    model.add(MaxPooling2D((3,3), strides=(1,1)))
    
    model.add(Conv2D(24, (3,3), padding='same', activation='relu', kernel_initializer='uniform'))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.5))
    model.add(MaxPooling2D((3,3), strides=(1,1)))

    model.add(Conv2D(12, (3,3), padding='same', activation='relu', kernel_initializer='uniform'))
    model.add(MaxPooling2D((3,3), strides=(1,1)))
    
    model.add(Flatten())
#     model.add(Dense(256, activation='relu'))
#     model.add(Dropout(0.25))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(6, activation='softmax'))

    return model

In [104]:
# model = init_model()
print(model.summary())
adam = keras.optimizers.Adam(lr=0.00001)#, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_90 (Conv2D)           (None, 21, 21, 48)        36912     
_________________________________________________________________
max_pooling2d_69 (MaxPooling (None, 19, 19, 48)        0         
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 19, 19, 24)        10392     
_________________________________________________________________
max_pooling2d_70 (MaxPooling (None, 17, 17, 24)        0         
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 17, 17, 12)        2604      
_________________________________________________________________
max_pooling2d_71 (MaxPooling (None, 15, 15, 12)        0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 2700)              0         
__________

In [109]:
batch_size = 128
epochs = 60

model.fit(images, labels_onehot,
              batch_size=batch_size,
              epochs=epochs,
              validation_split = 0.2,
              shuffle=True)

Train on 57600 samples, validate on 14400 samples
Epoch 1/60
57600/57600 [==============================] - 9s 159us/step - loss: 0.4900 - acc: 0.7955 - val_loss: 0.4949 - val_acc: 0.7994
Epoch 2/60
57600/57600 [==============================] - 9s 158us/step - loss: 0.4829 - acc: 0.8009 - val_loss: 0.5387 - val_acc: 0.7868
Epoch 3/60
57600/57600 [==============================] - 9s 161us/step - loss: 0.4921 - acc: 0.7944 - val_loss: 0.5662 - val_acc: 0.7667
Epoch 4/60
57600/57600 [==============================] - 9s 158us/step - loss: 0.4964 - acc: 0.7948 - val_loss: 0.5033 - val_acc: 0.7955
Epoch 5/60
57600/57600 [==============================] - 9s 158us/step - loss: 0.4874 - acc: 0.7981 - val_loss: 0.5348 - val_acc: 0.7835
Epoch 6/60
57600/57600 [==============================] - 9s 159us/step - loss: 0.4803 - acc: 0.8011 - val_loss: 0.5570 - val_acc: 0.7797
Epoch 7/60
57600/57600 [==============================] - 9s 158us/step - loss: 0.4803 - acc: 0.8020 - val_loss: 0.4914 - 

57600/57600 [==============================] - 9s 151us/step - loss: 0.4471 - acc: 0.8187 - val_loss: 0.4675 - val_acc: 0.8177
Epoch 60/60
57600/57600 [==============================] - 9s 153us/step - loss: 0.4413 - acc: 0.8194 - val_loss: 0.4568 - val_acc: 0.8224


In [107]:
model.save('xjbp.h5')